In [1]:
INPUT_FILE = '../data/processed/enriched_df.csv'
POPULATION_FILE = '../data/raw/total_pop_1960_2018.csv'
META_DATA = '../data/raw/meta_data.csv'

In [10]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import cufflinks as cf
cf.go_offline()

%matplotlib inline

In [3]:
%load_ext autoreload
%autoreload 2

## Read in data

In [4]:
df = pd.read_csv(INPUT_FILE)
# df_population = pd.read_csv(POPULATION_FILE, skiprows=4)

In [5]:
df.head()

,country,year,sex,age,suicides_no,population,fraction_pop,total_population
0,Albania,1985,female,15-24 years,NaN,277900.0,0.088361,2964762.0
1,Albania,1985,female,25-34 years,NaN,246800.0,0.083171,2964762.0
2,Albania,1985,female,35-54 years,NaN,267500.0,0.134128,2964762.0
3,Albania,1985,female,5-14 years,NaN,298300.0,0.089561,2964762.0
4,Albania,1985,female,55-74 years,NaN,138700.0,0.084270,2964762.0


## Plot some graphs

In [29]:
df_suicides = df.groupby(['year', 'sex'])['suicides_no'].sum().reset_index()

In [30]:
df_suicides = df_suicides.pivot(index='year', columns='sex', values='suicides_no')
df_suicides.head()

sex,female,male
year,,
1979,29418.0,63372.0
1980,42662.0,110886.0
1981,47530.0,130484.0
1982,47681.0,135212.0
1983,34823.0,87607.0


In [32]:
df_suicides.iplot(kind='bar')